<a href="https://colab.research.google.com/github/Firojpaudel/Demystifying_Language_Modeling/blob/main/Computes/Pytorch_and_FLOPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Floating Point Precisions:
---



Before diving into the tensors, first lets try to learn the GPU architectures as well.

Here, we will be comparing two popular GPUs, one in terms of SOTA performance and another in terms of availability.

In terms of SOTA performance, we have Nvidia's **H100** GPU. It is buit on Hooper Architecture. And was released on 2022.

**Some of its features:**
1. It has $80\text{GB HBM3}$ memory. (**HBM3** is the fourth generation of High Bandwidth Memory technology)
2. Bandwidth: $3.35 \text{ TB/s}$
3. Its optimised  for massive models (e.g., LLMs, diffusion models) with high memory and compute demands.

> **Note ⓘ** \
> These are not provided in Colab's Free Tier. However Pro Tier has one.


Now, lets go to **T4** GPU.

**T4** is based on Turing Architecture.

1. Release Date: 2018
2. Built for energy-efficient inference and lightweight ML workloads in data centers.
3. $16\text{GB GDDR6}$ memory, $320 \text{ GB/s}$ memory bandwidth, no NVLink

---

Also, while we are at it, let's compare $\text{TFLOPs}$ between these two:


|Precision | H100 (SXM, 80GB) | T4 (16 GB) |
|----------|------------------|------------|
|FP32	|67 TFLOPS	|8.1 TFLOPS|
|FP16	|989 TFLOPS (Tensor Cores)	|65 TFLOPS (Tensor Cores)|
|BF16	|989 TFLOPS (Tensor Cores)	|Not natively supported (emulated via FP16)|
|FP8	|1979 TFLOPS (Transformer Engine)	|Not supported|



> **Takeaway ⩩** \
> H100’s vastly superior TFLOPS across all precisions, especially FP8 and BF16, makes it ideal for SOTA AI workloads, while T4’s modest FP16/FP32 performance suits lighter tasks.
---

Now that we know a bit about the architectures, lets now understand Tensor Data Types (Precision) —code based :)

> **Note ⓘ** \
> Since we wil be using Colab free tier GPU (T4) the entire time, we will be sticking with all the formats that are compatible with this GPU only.

In [ ]:
##@